In [1]:
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_extraction.text import TfidfVectorizer


In [2]:
csv_path = "D:\\Reddit_Comment_Project\\archive\\final_clean_5.csv"

df = pd.read_csv(csv_path)

display(df.head(20))

,comment_id,score,subreddit,post_id,author_name,controversiality,user_total_karma,post_score,post_upvote_ratio,self_text,post_title
0,cm3dnw7,1,RussiaDenies,2m33hb,Player276,0,75258.0,5,0.57,lol title can not deny accusation,russia deny nato accusation troop ukraine
1,cmqs3d9,1,RussiaDenies,2ov646,Martenz05,0,49520.0,5,0.62,article estonian english translation ...,russia deny violate estonian airspace english ...
2,cmv6omo,2,RussiaDenies,2p9hv1,CanadaDry95,0,7071.0,14,0.75,putin outsmart gt everybody,russias defense ministry sunday deny swedish...
3,codh5tc,1,UkrainePics,2ueefh,ms_kat_d,0,4046.0,20,1.00,highly idyllic stuff nice,carpathian mountain western ukraine vlad sok...
4,covo4um,3,UkrainePics,2x06n4,I_AM_STILL_A_IDIOT,0,901387.0,18,0.93,thank yougallowboob share album,unfortunate reality war eastern ukraine ...
5,cp2t1z0,3,RussiaDenies,2xrsz3,MaltyBeverage,0,49800.0,17,0.87,russia confirm putin awesome russia confirm ...,russia deny cameras nemtsovs murder
6,cpv150u,2,RussiaDenies,30r8qq,The_Good_Stuffs,0,65.0,6,0.75,tend believe russians even hard b...,russia deny air traffic controller fault k...
7,cxd26dh,4,RussiaDenies,3u39er,Xecutor,0,37976.0,46,0.88,russia deny title russia deny innocent,russia innocent
8,cyg2591,6,RussiaDenies,3yq1af,chetmanly2,0,7065.0,11,1.00,russia also deny finland,russia deny claim syriabound missile fall iran
9,d26j63i,1,RussiaDenies,4f7hcs,autotldr,0,3973420.0,17,0.95,well tldr could make reduce bot gt...,russia deny claim jet barrelrolle us reco...


In [3]:
# Fill NaN values with 'no text'
df['self_text'].fillna('no text', inplace=True)

# Fill NaN values with 'no text'
df['post_title'].fillna('no text', inplace=True)

import nltk
nltk.download('sentiwordnet')
nltk.download('wordnet')


In [17]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import sentiwordnet as swn
from scipy.sparse import csr_matrix

# Step 1: Initialize TF-IDF Vectorizer
tfidf_vectorizer_self_text = TfidfVectorizer()

# Step 2: Fit and Transform
tfidf_matrix_self_text = tfidf_vectorizer_self_text.fit_transform(df['self_text'])

# Step 3: Convert to DataFrame
tfidf_df_self_text = pd.DataFrame(tfidf_matrix_self_text.toarray(), columns=tfidf_vectorizer_self_text.get_feature_names_out())

# Step 4: Define Function to Get Word Sentiment
def get_word_sentiment(word):
    synsets = list(swn.senti_synsets(word))
    if len(synsets) > 0:
        # Consider only the first synset (most common meaning)
        return synsets[0].pos_score() - synsets[0].neg_score()
    else:
        return 0.0  # If no synsets found, return neutral score

# Step 5: Define Function to Calculate Document Sentiment
def calculate_document_sentiment(tfidf_vector, feature_names):
    document_sentiment = 0.0
    total_weight = 0.0
    for idx, weight in enumerate(tfidf_vector):
        word = feature_names[idx]
        sentiment = get_word_sentiment(word)
        document_sentiment += weight * sentiment
        total_weight += weight
    if total_weight == 0:
        return 0.0  # Avoid division by zero
    return document_sentiment / total_weight

# Step 6: Apply SentiWordNet to TF-IDF vectors for self_text
sentiment_scores = []
for idx in range(tfidf_matrix_self_text.shape[0]):
    row = tfidf_matrix_self_text.getrow(idx)
    sentiment_score = calculate_document_sentiment(row.toarray()[0], tfidf_vectorizer_self_text.get_feature_names_out())
    sentiment_scores.append(sentiment_score)

# Add sentiment scores to DataFrame
df['self_text_sentiment_swn'] = sentiment_scores

# Display the DataFrame with sentiment scores
print(df[['self_text', 'self_text_sentiment_swn']])


KeyboardInterrupt: 

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import sentiwordnet as swn

# Step 1: Initialize TF-IDF Vectorizer
tfidf_vectorizer_post_title = TfidfVectorizer()

# Step 2: Fit and Transform
tfidf_matrix_post_title = tfidf_vectorizer_post_title.fit_transform(df['post_title'])

# Step 3: Convert to DataFrame
tfidf_df_post_title = pd.DataFrame(tfidf_matrix_post_title.toarray(), columns=tfidf_vectorizer_post_title.get_feature_names_out())

# Step 4: Define Function to Get Word Sentiment
def get_word_sentiment(word):
    synsets = list(swn.senti_synsets(word))
    if len(synsets) > 0:
        # Consider only the first synset (most common meaning)
        return synsets[0].pos_score() - synsets[0].neg_score()
    else:
        return 0.0  # If no synsets found, return neutral score

# Step 5: Define Function to Calculate Document Sentiment
def calculate_document_sentiment(tfidf_vector, feature_names):
    document_sentiment = 0.0
    total_weight = 0.0
    for idx, weight in enumerate(tfidf_vector):
        word = feature_names[idx]
        sentiment = get_word_sentiment(word)
        document_sentiment += weight * sentiment
        total_weight += weight
    if total_weight == 0:
        return 0.0  # Avoid division by zero
    return document_sentiment / total_weight

# Step 6: Apply SentiWordNet to TF-IDF vectors for post_title
def calculate_title_sentiment(comment_titles):
    sentiment_scores = []
    for idx in range(comment_titles.shape[0]):
        row = comment_titles.getrow(idx)
        sentiment_score = calculate_document_sentiment(row.toarray()[0], tfidf_vectorizer_post_title.get_feature_names_out())
        sentiment_scores.append(sentiment_score)
    return sentiment_scores

# Group by post_id and aggregate post titles
grouped_titles = df.groupby('post_id')['post_title'].apply(lambda x: ' '.join(x))

# Calculate sentiment scores for aggregated titles
title_sentiment_scores = calculate_title_sentiment(tfidf_vectorizer_post_title.transform(grouped_titles))

# Add sentiment scores to DataFrame
df['post_title_sentiment_swn'] = df['post_id'].map(grouped_titles.to_dict())
df['post_title_sentiment_swn'] = title_sentiment_scores

# Display the DataFrame with sentiment scores
print(df[['post_title', 'post_title_sentiment_swn']])
